# Process - SocialDF

In [33]:
import polars as pl
import re
from src.utils.download_insta_vids import download_insta_vids


real = pl.read_csv("../socialdf_real.csv").rename({"officialblazequotes\n":"url"}).with_columns(pl.lit(0).alias("target"))
fake = pl.read_csv("../socialdf_deepfake.csv").with_columns(pl.lit(1).alias("target")).rename({"DEEPFAKE DATA LINKS":"url"})
real.head()

url,target
str,i32
"""https://www.instagram.com/reel…",0
"""https://www.instagram.com/p/DB…",0
"""https://www.instagram.com/p/Cx…",0
"""https://www.instagram.com/p/Cs…",0
"""https://www.instagram.com/p/DE…",0


In [ ]:
df = pl.concat([real, fake])
ids = []
urls = df["url"].to_list()
for u in urls:
    try:
        match = re.search(r"instagram\.com/p/([a-zA-Z0-9_-]+)", u)
        ids.append(match.group(1))
    except:
        ids.append(None)
        
df = df.with_columns(pl.Series("id", ids)).filter(pl.col("id").is_not_null())


In [ ]:
urls = df["url"].to_list()
for u in urls:
    download_insta_vids(u, output_folder="vids")

In [ ]:
import os
ids_downloaded = [f.split("_")[0] for f in os.listdir("vids") if f.endswith(".mp4")]
df = df.filter(pl.col("id").is_in(ids_downloaded))
df.write_csv("socialdf_processed.csv")
df.group_by("target").count()